In [ ]:
from IPython.core.display import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Draw inline
%matplotlib inline

# Set figure aesthetics
sns.set_style("white", {'ytick.major.size': 10.0})
sns.set_context("talk", font_scale=1.1)

In [ ]:
train_users = pd.read_csv('input/train_users_2.csv')
test_users = pd.read_csv('input/test_users.csv')
piv_train = train_users.shape[0] # Store length of train_users file
labels = train_users['country_destination'].values

all_users = pd.concat([train_users,test_users],axis=0,ignore_index=True) #Combine train and test for joint processing.
all_users_preserved = all_users

In [ ]:
session_df = pd.read_csv('input/sessions.csv')
session_users = session_df

# Clean missing values
session_users['secs_elapsed'].fillna(0,inplace=True)
session_users['action_type'].fillna('',inplace=True)

session_df.head(20)

In [ ]:
## Okay let's try adding all the groupings separately.
session_action = session_users.groupby(['user_id','action']).count()['secs_elapsed'].unstack().fillna(0)
session_action_type = session_users.groupby(['user_id','action_type']).count()['secs_elapsed'].unstack().fillna(0)
session_action_detail = session_actions = session_users.groupby(['user_id','action_detail']).count()['secs_elapsed'].unstack().fillna(0)

sessions_pure = pd.merge(session_action,session_action_type,left_index=True,right_index=True,how='inner')
sessions_pure = sessions_pure.merge(session_action_detail,left_index=True,right_index=True,how='inner')

In [ ]:
# Create New Combined Feature for Grouping
session_users['action_action_detail'] = session_users['action']+'__'+session_users['action_detail']
session_users2 = session_users.groupby(['user_id','action_action_detail']).count()['secs_elapsed'].unstack().fillna(0)
print('Number of Action Count Features created:', session_users2.shape[1])
print('\n')

In [ ]:
# Time spent on each Device
deviceusage_df = session_df[session_df['secs_elapsed']<60*100][['user_id','device_type','secs_elapsed']].groupby(['user_id','device_type']).sum().unstack()['secs_elapsed']
# Number of device types used
deviceusage_df['num_devices'] = session_df.groupby('user_id')['device_type'].agg(lambda x: len(np.unique(x.values)))
deviceusage_df.fillna(0,inplace=True)
deviceusage_df.head(2)

In [ ]:
## Features about individual sessions (sittings)
in_session_actions = session_df[session_df['secs_elapsed']<100*60].groupby('user_id')
AA = in_session_actions['secs_elapsed'].agg({'total_session_times': np.sum,
                                        'mean_secs_per_action' : np.mean,
                                            'num_actions_insession' : np.size})

## Features related to the number of sessions 
between_session_actions = session_df[session_df['secs_elapsed']>200*60].groupby('user_id')
BB = between_session_actions['secs_elapsed'].agg({'num_sessions': np.size,
                                            'mean_between_sessions' : np.mean,
                                            'max_between_sessions': np.max
                                            })

session_times = pd.merge(AA,BB,how='outer',left_index=True,right_index=True)

#Count number of different days spent on AirBnB. Obtained by looking for number of breaks longer than 18 hours.
session_times['num_days'] = session_df[session_df['secs_elapsed']>18*3600].groupby('user_id').count()['secs_elapsed']
session_times.fillna(0,inplace=True)
session_times['num_sessions'] += 1 # Doesn't really make a difference
session_times['num_days'] += 1 
session_times['all_time'] = session_df.groupby('user_id').sum()['secs_elapsed']

session_times.head(3)

In [ ]:
## Need to attach labels for feature selection.
SessionX = pd.merge(train_users[['id','country_destination']],sessions_pure,right_index=True,left_on='id',how='inner')
Sessiony = SessionX['country_destination']!="NDF" # Turn this into a binary classification problem
SessionX.drop(['id','country_destination'],axis=1,inplace=True)


# Perform selection
from sklearn.feature_selection import SelectKBest
selector = SelectKBest(k=300)
selector.fit(SessionX,Sessiony)
sessions_pure = sessions_pure[sessions_pure.columns[selector.get_support()]]

# Examine feature selection univariate statistics test results:
FeaturePvalues = pd.Series(index=SessionX.columns,data=selector.pvalues_).sort_values(ascending=True)
FeatureScores = pd.Series(index=SessionX.columns,data=selector.scores_).sort_values(ascending=False)
# Plot
k = 50
plt.figure(figsize=(10,k/3))
sns.barplot(y=FeatureScores.index[0:k],x=FeatureScores.head(k))
plt.title('F-Anova Values for Session Action Count Features: Top 20')

In [ ]:
A = session_df.groupby('user_id').cumsum()
A['user_id'] = session_df['user_id']

b = A['secs_elapsed'].values
A['Diff'] = -(b-(np.append(b[1:],0)))
A['Diff'] = np.insert(A['Diff'].values[:-1],0,0)
A.loc[A['Diff']>100*60,'Diff']=0
A.loc[A['Diff']<1,'Diff']=0


A['Week'] = A['secs_elapsed']//(86400*7)
A['CumDiff'] = A.groupby(['user_id','Week']).cumsum()['Diff']/3600
WeekFeatures = A.groupby(['user_id','Week']).sum()['Diff']
WeekFeatures = WeekFeatures.unstack()[np.arange(0,6)].fillna(0)//3600

In [ ]:
WeekFeatures.head()

In [ ]:
## Join all session features together
SessionFeatures = pd.merge(sessions_pure,deviceusage_df,right_index=True,left_index=True,how='inner')
SessionFeatures = pd.merge(SessionFeatures,session_times,right_index=True,left_index=True,how='inner')
SessionFeatures = pd.merge(SessionFeatures,WeekFeatures,right_index=True,left_index=True,how='inner')
print('Total number of Session Features created:', SessionFeatures.shape[1])

In [ ]:
SessionFeatures.head()

In [ ]:
X = pd.merge(train_users[['id','country_destination']],SessionFeatures,right_index=True,left_on='id',how='inner')
y = X['country_destination']!="NDF" # Turn this into a binary classification problem
X.drop(['id','country_destination'],axis=1,inplace=True)

In [ ]:
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn import preprocessing

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X = X.loc[:,np.arange(0,6)]

In [ ]:
X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [ ]:
RF = RandomForestClassifier(oob_score=True,n_estimators=100,n_jobs=-1)
RF.fit(X_train,y_train)
y_pred = RF.predict(X_test)

In [ ]:
print(metrics.classification_report(y_test,y_pred))


In [ ]:
1 - sum(y)/len(y)

In [ ]:
metrics.accuracy_score(y_pred,y_test)

In [ ]:
sum(y_pred)/len(y_pred)

In [ ]:
FeatureImportances = pd.Series(index=X.columns,data=RF.feature_importances_).sort_values(ascending=False)
k = 100
plt.figure(figsize=(10,20))
sns.barplot(y=FeatureImportances.index[0:k],x=FeatureImportances.head(k))

In [ ]:
Sessions = SessionX

In [ ]:
session_df

In [ ]:
session_df.set_index('d')

In [ ]:
A

In [ ]:
A.head()

In [ ]:
a = WeekFeatures.unstack().count(axis=0)
a.cumsum()/a.sum()

In [ ]:
.groupby('user_id').cumsum()

In [ ]:
Sessions['Booked']=Sessiony

In [ ]:
Sessions

In [ ]:
sns.countplot(data=Sessions,hue='Booked',x='booking_request')

In [ ]:
# P(Booked|booking request)
sum((Sessions['booking_request']>0)&Sessions['Booked'])/sum(Sessions['booking_request']>0)

In [ ]:
# P(Booked|-booking request)
sum((Sessions['booking_request']==0)&Sessions['Booked'])/sum(Sessions['booking_request']==0)

In [ ]:
for i in range(0,50):
    I = FeatureScores.index[i]
    temp1 = sum((Sessions[I]>0)&Sessions['Booked'])/sum(Sessions[I]>0)
    print('P(Booked|',I,')=',temp1)
    temp2 = sum((Sessions[I]==0)&Sessions['Booked'])/sum(Sessions[I]==0)
    print('P(Booked|',I,')=',temp2)
    print('Odds Ratio: ',temp1/temp2)
    print('\n')

In [ ]:
FeatureScores.index[0]

In [ ]:
FeatureScores

In [ ]:
ST = pd.merge(train_users[['id','country_destination']],session_times,right_index=True,left_on='id',how='inner')
ST['Booked'] = ST['country_destination']!='NDF'

In [ ]:
Sessions.plot.

In [ ]:
Sessions[list(FeatureScores.index[0:2])+['Booked']].plot.(by='Booked');

In [ ]:
ST.head(2)

In [ ]:
sns.set_style("whitegrid")


In [ ]:
sns.countplot(hue="Booked",x="num_days",data=ST,order=range(0,20))
#plt.ylim([0,20])

In [ ]:
sns.countplot(hue="Booked",x="days_all",data=ST,order=range(0,20))
#plt.ylim([0,20])

In [ ]:
np.round(session_times['total_session_times']/(3600))

In [ ]:
ST = pd.merge(train_users[['id','country_destination']],session_times,right_index=True,left_on='id',how='inner')
ST['Booked'] = ST['country_destination']!='NDF'
ST

In [ ]:
ST['total_session_times'] = np.round(ST['total_session_times']/(3600))
ST.head(5)

In [ ]:
ST

In [ ]:
ST.sort_values(by='total_session_times',ascending=False)

In [ ]:
STTS = ST.groupby(['total_session_times','Booked']).count()['id'].unstack().fillna(0)
STTS['Proportion'] = STTS[True]/(STTS[True]+STTS[False])
STTS['Total'] = (STTS[True]+STTS[False])
STTS['CumProp'] = STTS[True].cumsum()/STTS[True].sum()

In [ ]:
STP.describe()

In [ ]:
ST.describe()

In [ ]:
STTS[True].cumsum().plot()
plt.xlim([0,100])

In [ ]:
STTS['CumProp'].plot()
plt.xlim([0,100])

In [ ]:
STTS['Proportion'].plot()
plt.xlim([0,60])

In [ ]:
STTS[STTS.columns[0:2]].plot()
plt.xlim([0,60])

## For people we are predicting correctly, how much info do we have?

# What useful things could we do here?
### Identify who the at risk users who have been looking for a long time and otherwise have the profile of someone who would book? Identify pain points . . .

#### By that point we know where they're interested in booking. Might want to ask questions about is it a matter of price? Is it a matter of availability?

#### Find people who are dissatisfied with price in one region and find another . . .

In [ ]:
ST['days_all'] = np.round(ST['all_time']/(3600*24))
STP = ST.groupby(['days_all','Booked']).count()['id'].unstack().fillna(0)
STP['Proportion'] = STP[True]/(STP[True]+STP[False])
STP['Total'] = (STP[True]+STP[False])
STP['CumProp'] = STP[True].cumsum()/STP[True].sum()

In [ ]:
STTS

In [ ]:
import matplotlib
matplotlib.style.use('ggplot')


In [ ]:
STP['days_all'] = np.round(ST['all_time']/(3600*24))
STP = ST.groupby(['days_all','Booked']).count()['id'].unstack().fillna(0)
STP['Proportion'] = STP[True]/(STP[True]+STP[False])
STP['Total'] = (STP[True]+STP[False])
STP['CumProp'] = STP[True].cumsum()/STP[True].sum()

In [ ]:
STP[True].sum()

In [ ]:
STP[True].cumsum().loc[100]

In [ ]:
STP[True].cumsum().plot()
plt.xlim([0,100])

In [ ]:
STP['CumProp'].plot()
plt.xlim([0,100])

In [ ]:
STP['Proportion'].plot()
plt.xlim([0,60])

In [ ]:
STP[STP.columns[0:2]].plot(kind='bar')
plt.xlim([0,30])

In [ ]:
STP[STP.columns[0:2]].plot()
plt.xlim([0,30])

In [ ]:
## Can rerun this on hours of site usage.

In [ ]:
## Data Quality

In [ ]:
ST

In [ ]:
sns.boxplot(x="Booked",y="num_days",data=ST)
plt.ylim([0,20])

Bin according to number of days . . .
Then calculate proportion.

In [ ]:
ST

In [ ]:
ST.groupby(['id',])

In [ ]:
ST['num_days'].quantile(.98)

In [ ]:
sns.boxplot(x="Booked",y="num_days",data=ST)
plt.ylim([0,23])

In [ ]:
sns.boxplot(x="Booked",y="booking_request",data=Sessions)

In [ ]:

plt.boxplot(x=ST['num_days'],y='Booked')

In [ ]:
g = sns.factorplot(x='num_days',hue='Booked',data=ST,kind='bar')

In [ ]:
ST

In [ ]:
sessions_pure